## Exercise 2: El Gamal Encryption and Signatures


_CSC596/686-G: The Real Block Chain_

_semester 211_

_burton rosenberg_

_15 october 2020_



### Exponents and discrete logs

Security in cryptography is about the ability or inability to solve mathematicaly puzzles quickly. The problem featured by El Gamal type public key cryptography is the difficulty gap between exponentiation,

$$
x \rightarrow g^x\pmod{p}
$$

and its inverse function, the discrete logarithm,

$$
g^x \rightarrow x \pmod{p}
$$

_Note 1:_ The arithmetic is modulo a prime $p$. Because $p$ is prime, there must be a $g$ such that the equation,

$$
g^x = y \pmod{p}
$$

has a solution for every  non-zero $y$, and that solution will be unique. So exponentiation/discrete log make a pairing, 

$$
\{\, (x,y) \,|\, g^x=y \bmod{p}\,\}
$$

That cryptography uses to create hard puzzles that can be solved with insider information.



### Exercise 1: Exponentiation is fast

There is an efficient way to calculate $g^x\bmod{p}$. It is not efficient to multiply $g$ to inself $x$ times, because a short $x$, say six digits$, make for a lot of work, up to one million multiplications, in case of six digits.

It is efficient to calculate this way,

$$
g^x = \begin{cases}
1 & \mbox{ if } x=0\\
g \, (g^{x-1}) &\mbox{ if $x$ is odd}\\
(g^{x/2})^2  &\mbox{ if $x$ is even}
\end{cases}
$$

Complete the following code that calculates how many multiplications it would take to computer $g^x$ using this recursion,

In [1]:

def how_many_mult(x):
    if x==0:
        # this is not the correct value to return
        return None
    if x%2==1:
        # this is not the correct value to return
        return how_many_mult(0)
    else:
        # this is not the correct value to return
        return how_many_mult(0)    

def test_how_many_mult():
    test_in = [0,1,2,3,4,10,20,30,100,1000,10000,1234567,234567891]
    test_out = [0, 1, 2, 3, 3, 5, 6, 8, 9, 15, 18, 31, 45]
    print(f'x:\t# mult\tcorrect?')
    correct = True
    for i in range(len(test_in)):
        x = test_in[i]
        n = how_many_mult(x)
        t = n==test_out[i]
        print(f'{x}:\t{n}\t{t}')
        correct = correct and t
    if correct:
        print(f'verdict: OK')
    else:
        print(f'verdict: fail')
    return

test_how_many_mult()

x:	# mult	correct?
0:	None	False
1:	None	False
2:	None	False
3:	None	False
4:	None	False
10:	None	False
20:	None	False
30:	None	False
100:	None	False
1000:	None	False
10000:	None	False
1234567:	None	False
234567891:	None	False
verdict: fail


### Exercise 2: Logarithms are slow

Run the following code and consider the run time. What size prime would you recommend so that the logarithm problem is so slow that your encryptions are safe?

What are you assuming about the computers available to the adversary?

Do you think a faster language will help? How about GPU's?

Can you estimate the power consumption required to do these calculations, and base your primes on the known available power production of a region or country? [NSA Power Consumption](https://en.wikipedia.org/wiki/National_Security_Agency#Power_consumption)


In [2]:
import time
import math

def test_speed(up_to_digits):
    """
    time N up to size N, where N = 10^n, i.e. n. digits long
    """
    for n in range(1,up_to_digits):
        g = 2
        N = 10**n
        
        ## time to do an exponentiation
        d = math.ceil(math.log(N,2))
        t_exp = time.clock()
        for i in range(d):
            g = (g*g)%N
        t_exp = time.clock()-t_exp
           
        ## time to do a logarithm
        t_log = time.clock()
        for i in range(N):
            g = (g*g)%N
        t_log = time.clock()-t_log
        
        print(f'{n} digits:\n\texp: {t_exp:.8f} sec\n\tlog: {t_log:.4f} sec')
    print('Done!')
        
#test_speed(11)

### Exercise 3: Finding the log

Complete the code for the log method. Given a $y$, try all values of $x$ between 0 and $p-1$. There should be exactly one value of $x$ for which $g^x=y\bmod{p}$. Return the value. 

The test function has the answer generators $2$ and $7$. You must past that test.


In [3]:

class ModularArithmetic:
    
    def __init__(self,p,generator=0,is_sophiegermain=False):
        """
        p is a prime, 
        find a generator g
        """
        self.p = p
        self.is_sg = is_sophiegermain
        self.g = 0
        
    def pwr(self,x,y):
        """
        return x^y % p
        this is done recursively, to show
        it can be done efficiently.
        
        x^y = (x^{y/2})^2 if y even, 
              else = x(x^{y-1})
              
        """
        if y==0: return 1
        if y%2==1:
            return (x*self.pwr(x,y-1))%self.p
        z = self.pwr(x,y//2)
        return (z*z)%self.p

    def exp(self,x):
        '''
        exp takes the argument to the power the selected generator
        of the group
        '''
        assert(self.g!=0)
        x = x%self.p
        return self.pwr(self.g,x)

    ## to be completed by the student
    def log(self,x):
        """
        the inverse function of exp
        
        this cannot be done efficiently.
        just try all y until g^y==x
        """
        assert(self.g!=0)
        if x<=1: return 0
       
        # calculate the log
        pass
    
        return None

    def inv(self,x):
        """
        uses little fermat to compute the inverse of x mod p
        x^{p-1}=1, so x^{-1} = x^{p-2}
        """
        if x==0: return 0
        x = x%self.p
        return self.pwr(x,self.p-2)

    def inv_sg(self,x):
        """
        this is somewhat obscure. if p is a safe prime,
        (p-1)/2 is a prime (a sophie germain prime)
        and we can calculate its inverse using little fermat
        """
        assert(self.is_sg)
        sgp = (self.p-1)//2
        x = x%(self.p-1)
        return (x**(sgp-2))%(self.p-1)

    def mul(self,x,y):
        return (x*y)%self.p
    
    def add(self,x,y):
        return (x+y)%self.p
   
    def is_gen(self,g):
        """
        checks if g is a generator by checking the order.
        """
        o = self.order(g)
        if o==(self.p-1):
            return True
        return False

    def find_generator(self):
        for g in range(2,self.p):
            if self.is_gen(g):
                self.g = g
                break

    def find_generator_answer(self):
        for g in range(2,self.p):
            if self.is_gen(g):
                self.g = g
                break

    def order(self,g):
        """
        returns the order of g in the integers mod p
        """
        if g==0: return 0
        if g==1: return 1
        o = g
        i = 1
        while o!=1:
            o = (g*o)%self.p
            i += 1
        return i
        
    def print_all(self):
        print(f'prime:\t{self.p}\ngen:\t{self.g}')
        

In [4]:

def test_inv():
    prime = 13
    ans = [1, 7, 9, 10, 8, 11, 2, 5, 3, 4, 6]
    ma = ModularArithmetic(prime) 
    for n in range(1,ma.p-1):
        assert(ans[n-1]==ma.inv(n))
    print('*** test_inv: ok ***')
    return True

def test_exp():
    prime = 13
    ma = ModularArithmetic(prime)
    
    ans = {2:[2, 4, 8, 3, 6, 12, 11, 9, 5, 10, 7, 1],
           7:[7, 10, 5, 9, 11, 12, 6, 3, 8, 4, 2, 1]}
    for ma.g in [2,7]:
        for n in range(1,ma.p):
            assert(ans[ma.g][n-1]==ma.exp(n)) 
    print('*** test_exp: ok ***')
    return True

def test_log():
    prime = 13
    ma = ModularArithmetic(prime)
    
    ans = {2:[0, 1, 4, 2, 9, 5, 11, 3, 8, 10, 7, 6],
           7:[0, 11, 8, 10, 3, 7, 1, 9, 4, 2, 5, 6]}
    t = True
    for ma.g in [2,7]:
        for n in range(1,ma.p):
            try:
                assert(ans[ma.g][n-1]==ma.log(n)) 
            except AssertionError:
                print(f'log wrong for {n} to base {ma.g}')
                t = False
    print(f'*** test_log: {t} ***')
    return True

def test_gen():
    prime = 13
    ans = [1, 12, 3, 6, 4, 12, 12, 4, 3, 6, 12, 2]
    ma = ModularArithmetic(prime)
    for n in range(1,ma.p):
        assert(ans[n-1]==ma.order(n))
    print('*** test_gen: ok ***')
    return True

test_inv()
test_exp()
test_log()
test_gen()

*** test_inv: ok ***
*** test_exp: ok ***
log wrong for 2 to base 2
log wrong for 3 to base 2
log wrong for 4 to base 2
log wrong for 5 to base 2
log wrong for 6 to base 2
log wrong for 7 to base 2
log wrong for 8 to base 2
log wrong for 9 to base 2
log wrong for 10 to base 2
log wrong for 11 to base 2
log wrong for 12 to base 2
log wrong for 2 to base 7
log wrong for 3 to base 7
log wrong for 4 to base 7
log wrong for 5 to base 7
log wrong for 6 to base 7
log wrong for 7 to base 7
log wrong for 8 to base 7
log wrong for 9 to base 7
log wrong for 10 to base 7
log wrong for 11 to base 7
log wrong for 12 to base 7
*** test_log: False ***
*** test_gen: ok ***


True

### Exercise 4: El Gamal Encryption

The trick of El Gamal ecryption is to create a way to use insider information to solve a hard puzzle involving a logarithm. 

The idea is Bob knows $x$ and Alice knows $k$, and they will agree on $x\, k$. They only have a public channel to communicate, so they do not communiate $x$ and $k$ but $g^x$ and $g^k$. Neither can retrive the other's secret, no one can, but that does not mean they cannot agree on $g^{k\,x}$,

- Bob computes $(g^{k})^{x}$ with what he knows, 
- and Alice computes $(g^{x})^{k}$ with what she knows. 

They have agreed on $x\,k$ without really knowing it. What they both know is $g^{x\,k}$, but that is just fine. Only they know this number, as it seems the only way to calculate it is to know $k$, or $x$, or both. And this excludes the entire world except Alice and Bob.

| Value | Who knows? | How they know |
|:-------:|:-----------|:---------------|
| $x$     |only Bob    | Bob chose it randomly |
| $k$     |only Alice    | Alice chose it randomly |
| $g^x$   |  everyone |  Bob announces his public key |
| $g^k$   | everyone |  Alice puts it in a public message |
| $g^{xk}$ | only Alice and Bob | Bob uses what Alice publicly said and his secret |
| | |   and Alice uses what Bob publicly said and her secret |



This seemingly random number $g^{x\,k}$, known in common by Alice and Bob, and only Alice and Bob, is used to hide the message, using multiplication mod $p$.

__Setup:__

1. Bob chooses a random $x$ in the range 1 through $p-1$.
2. Bob computes and announces publicly his public key $g^x$.
3. Note that all computation is done modulo $p$.

__To Encrypt:__

1. Alice chooses a random $k$ in the range 1 through $p-1$.
2. Alice computes $r = g^k$ and $\gamma = \beta^k$, with Bob's public key $\beta$.
3. Alice sends the pair $(r, \gamma m)$ to Bob over a public communication channel.
4. Note that all computation is done modulo $p$.

__To Decrypt:__

1. Bob receives $(r, \gamma \, m)$.
2. Bob computes $r^x$. Note that $r^x = (g^k)^x =(g^x)^k = \beta^k =  \gamma$.
3. Bob recovers the message, $\gamma^{-1}\gamma m = m$.
4. Note that all computation is done modulo $p$.


In [5]:
import random

class ElGamal:
    
    def __init__(self,p):
        """
        important, p and (p-1)/2 are both prime 
        p is called a safe prime, and (p-1)/2 is called a sophie
        germain prime
        p = 1907
        """
        self.ma = ModularArithmetic(p,is_sophiegermain=True)
        self.p = p
        self.g = 0
        self.public = 0
        self.private = 0 

        # set a generator
        self.ma.find_generator()
        # set a private/public key pair
        self.generate_key()
        
    def generate_key(self):
        self.private = random.randrange(2,self.p-2)
        self.public = self.ma.exp(self.private)
        return (self.private,self.public)
    
    
    ## student to complete
    def encrypt(self,m):
        """
        returns message m encrypted. note: the message is
        an integer between 0 and self.p-1. 
        
        the returned encrypted messages is a pair (r,s).
        """
        m = m%self.p
        k = random.randrange(1,self.p-1)
        
        pass
    
        return (0,0)
    
    ## student to complete
    def decrypt(self,c):
        """
        given an encrypted message c, a pair (r,s), decrypts
        and returns the messge.
        """
        (r,s) = c
        
        pass
    
        return 0
    
    def sign(self,m):
        k = 0
        m = m%(self.p-1)
        sgp = (self.p-1)//2
        while (k%2==0) or k%sgp==0 :
            k = random.randrange(2,self.p-2)
        k_inv = k**(sgp-2)%(self.p-1)
        # k_inv = self.ma.inv_sg(k)
        if (k*k_inv)%(self.p-1)!=1:
            print(f'k_inv fails: k: {k}, kinv: {k_inv}, p_sg: {(self.p-1)//2}')
        assert((k*k_inv)%(self.p-1)==1)
        r = self.ma.exp(k)
        u = (m  + (self.private*r)%(self.p-1))%(self.p-1)
        s = (k_inv*u)%(self.p-1)
        assert(self.ma.pwr(r,s)==self.ma.exp(u))
        return (r,s)
    
    def verify(self,sig,m):
        (r,s) = sig
        m = m%(self.p-1)
        v1 = self.ma.pwr(r,s)
        v2 = self.ma.mul(self.ma.exp(m),self.ma.pwr(self.public,r))
        return v2==v1
 

In [6]:
def test_elgamal_encryption(verbose=False):
    sophie_germain_prime = 1907
    eg = ElGamal(sophie_germain_prime)
    t = True
    for i in range(10):
        m = random.randrange(1,sophie_germain_prime)
        cipher_0 = eg.encrypt(m)
        for j in range(10): 
            cipher = eg.encrypt(m)
            m_out = eg.decrypt(cipher)
            if verbose: print(f'{i},{j}:\tmessage:{m}\n\tencryption: {cipher}')
            t = t and (m==m_out)
            t = t and (cipher!=cipher_0)
    print(f'*** test_elgamal_encryption: {t} ***')
    return t


def test_elgamal_signature(verbose=False):
    sophie_germain_prime = 1907
    eg = ElGamal(sophie_germain_prime)
    t = True
    for i in range(10):
        m = random.randrange(1,sophie_germain_prime)
        sig_0 = eg.sign(m)
        for j in range(10):
            sig  = eg.sign(m)
            t = t and eg.verify(sig,m)
            #t = t and (sig!=sig_0)
            if verbose: print(f'message:{m}\n\tsignature: {sig}')

    print(f'*** test_elgamal_signature: {t} ***')
    return t


test_elgamal_encryption(verbose=False)
test_elgamal_signature(verbose=False)



*** test_elgamal_encryption: False ***
*** test_elgamal_signature: True ***


True

### Exercise 5: El Gamal Signatures

The El Gamal signature scheme uses the same principle of computing in the exponent, in order to verify an equation without actually knowing values. The equation that will be created (privately) by the signer and verified (publicly) by the verifier is,

$$
s = k^{-1}(H(m) - x r) \pmod{p-1}
$$

where, $H$ is a hash function, $m$ is a message, $x$ is the signer's secret, and $k$ is a random value, chosen by the signer, secret to the signer, and never re-used, and $r = g^k\bmod{p}$ is the public commitment to $k$.

This equation is publicly verifiable because $r^s$ computes out the all pubicly known inputs, 

$$
r^s = (g^k)^{k^{-1}(H(m) - x r)} = g^{H(m)}\,(g^x)^{-r} =   g^{H(m)}\,(g^x)^{-r}
 =  g^{H(m)}\,\beta^{-r}
$$

__Note well:__ The change from modulo $p$ to modulo $p-1$ for computations in the exponent. Since there is no such $x$ that gives $g^x=0$, the exponents only work over the $p-1$ non-zero elements in the integers mod $p$, and obey,

$$
g^x = g^{x+d(p-1)} = g^x\,g^{d(p-1)} = g^x\, (g^{p-1})^d = g^x\,1^d = g^x \pmod{p}
$$

for any $d$. (See Fermat's Little Theorem.)


What does verification tell us? If a hash function $H$ is not used, it tells us nothing.

The verification equation is written,

$$
r^s \, \beta^{r} =g^{H(m)}
$$

Put this all in the base $\beta$ by calling out an $r = \beta^v$ and then adjust $s$ to make the exponent on the left zero,

$$
r^s \, \beta^{r} = (\beta^v)^s \, \beta^{r} = \beta^{vs+r}
$$

by solving $vs+r=0\bmod{p-1}$. This $(r,s)$ signs the message 0. Setting $r=\beta^v\,g^u$ extends this to sign the message $su$. The has function challenges the adversary to product an $H$-preimage of the value $su$.

Hence we have to cosider the value $H(m)$ as a public challenge, essentially a random number, that we must sign.

__Exercise:__ I have provided the implementation of El Gamal signatures for you, without a hash function.

Provide 6 forged signatures. To do this you will need to read the public key out of the ElGamal object, and re-insert that public key into the object when verfiying your forged signatures. It will not be necessary to peek at the private key.
